In [211]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, roc_auc_score
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/insurance/x_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/insurance/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/insurance/x_test.csv")

train = pd.merge(x_train, y_train, on='ID')
x = x_train.drop(['ID','id'], axis=1)
y = y_train['Response']
target = x_test.copy()

# print(x.shape)
# print()
# print(x.info())
# print()
# print(x.describe())
# print()
# print(x.isnull().sum())
# print()
# print(target.shape)
# print()
# print(target.info())
# print()
# print(target.describe())
# print()
# print(target.isnull().sum())

obj_col = x.select_dtypes(include='object').columns
num_col = x.select_dtypes(exclude='object').columns

# for col in obj_col:
#   print(col)
#   print(x[col].unique())
# print()
# for col2 in obj_col:
#   print(col2)
#   print(target[col2].unique())

x = pd.get_dummies(x, columns=obj_col, dtype='int')
target = pd.get_dummies(target, columns=obj_col, dtype='int')
target = target.reindex(columns=x.columns)

# mms = MinMaxScaler()
# x[num_col] = mms.fit_transform(x[num_col])
# target[num_col] = mms.transform(target[num_col])

rs= 810
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, stratify=y, random_state=rs)

model1 = RandomForestClassifier(random_state=rs)


param = {'n_estimators':[5,10,15,20],
         'max_depth':[5,10,15,20]}
gscv = GridSearchCV(estimator=model1, param_grid=param, cv=5)
gscv.fit(x_train, y_train)

print(gscv.best_estimator_)
model2 = gscv.best_estimator_

pred_val = model2.predict(x_val)
pred_val_proba = model2.predict_proba(x_val)[:,:1]

print('f1_score:', f1_score(y_val, pred_val))
print('auc_score:', roc_auc_score(y_val, pred_val_proba))

pred = model2.predict(target)
result = pd.DataFrame({'Id':x_test['ID'], 'pred':pred})
result.to_csv('result.csv', index=False)
print(result)

RandomForestClassifier(max_depth=10, n_estimators=15, random_state=810)
f1_score: 0.0
auc_score: 0.14588578123459572
           Id  pred
0          17     0
1          19     0
2          21     0
3          23     0
4          32     0
...       ...   ...
76217  381091     0
76218  381097     0
76219  381102     0
76220  381105     0
76221  381108     0

[76222 rows x 2 columns]


In [210]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, roc_auc_score

# 데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/insurance/x_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/insurance/y_train.csv")
x_test = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/insurance/x_test.csv")

train = pd.merge(x_train, y_train, on='ID')
x = x_train.drop(['ID', 'id', 'Response'], axis=1)
y = train['Response']
target = x_test.copy()

# 더미화 및 컬럼 정렬
obj_col = x.select_dtypes(include='object').columns
x = pd.get_dummies(x, columns=obj_col, dtype='int')
target = pd.get_dummies(target, columns=obj_col, dtype='int')
target = target.reindex(columns=x.columns, fill_value=0)

# Min-Max Scaling
num_col = x.select_dtypes(exclude='object').columns
mms = MinMaxScaler()
x[num_col] = mms.fit_transform(x[num_col])
target[num_col] = mms.transform(target[num_col])

# 데이터 분리
rs = 810
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, stratify=y, random_state=rs)

# 모델 및 하이퍼파라미터 튜닝
model1 = RandomForestClassifier(random_state=rs)
param = {'n_estimators': [5, 10, 15, 20], 'max_depth': [5, 10, 15, 20]}
gscv = GridSearchCV(estimator=model1, param_grid=param, cv=5, refit=True)
gscv.fit(x_train, y_train)

# 성능 평가
pred_val = gscv.predict(x_val)
pred_val_proba = gscv.predict_proba(x_val)[:, 1]

print('Best Parameters:', gscv.best_params_)
print('f1_score:', f1_score(y_val, pred_val))
print('auc_score:', roc_auc_score(y_val, pred_val_proba))

# 테스트 데이터 예측 및 저장
pred = gscv.predict(target)
result = pd.DataFrame({'Id': x_test['ID'], 'pred': pred})
result.to_csv('result.csv', index=False)
print(result.head())

Best Parameters: {'max_depth': 10, 'n_estimators': 15}
f1_score: 0.0
auc_score: 0.854083256415302
   Id  pred
0  17     0
1  19     0
2  21     0
3  23     0
4  32     0


In [ ]:
import pandas as pd
df =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/bicycle/seoul_bi.csv')
df.head()

,대여일자,대여시간,대여소번호,대여구분코드,성별,연령대코드,이용건수,운동량,탄소량,이동거리,사용시간
0,2021-06-01,0,3541,정기권,F,~10대,1,0.00,0.00,0.00,8
1,2021-06-01,0,765,정기권,F,~10대,1,27.21,0.35,1526.81,19
2,2021-06-01,0,2637,정기권,F,~10대,1,41.40,0.37,1608.56,18
3,2021-06-01,0,2919,정기권,F,~10대,1,0.00,0.00,0.00,75
4,2021-06-01,0,549,정기권,F,~10대,1,13.04,0.17,731.55,6


In [ ]:
ans = df.groupby('대여일자').size().sort_values(ascending=False)
print(ans.index[0])

2021-06-04


In [ ]:
df['대여일자'] = pd.to_datetime(df['대여일자'])
df['day_name'] = df['대여일자'].dt.day_name()

ans = df.value_counts('day_name')
print(ans)

day_name
Wednesday    110607
Friday       108877
Monday       107568
Tuesday      103704
Saturday     101299
Sunday        90918
Thursday      55977
Name: count, dtype: int64


In [ ]:
target = df.groupby(['day_name','대여소번호'], as_index=False).size().sort_values('size', ascending=False)
ans = target.drop_duplicates('day_name', keep='first').sort_values('day_name',ascending=False).reset_index(drop=True)
print(ans)

    day_name  대여소번호  size
0  Wednesday    502   282
1    Tuesday    502   267
2   Thursday   2715   137
3     Sunday    502   372
4   Saturday    502   378
5     Monday    502   242
6     Friday    502   277


In [ ]:
daily = df[df['대여구분코드'].isin(['일일권','일일권(비회원)'])].연령대코드.value_counts().sort_index()
total = df['연령대코드'].value_counts().sort_index()
print(total)
print(daily)

ans = (daily/total).sort_values(ascending=False)
print(ans)

연령대코드
20대     247561
30대     186722
40대     114799
50대      70428
60대      19288
70대~      3227
~10대     36925
Name: count, dtype: int64
연령대코드
20대     59393
30대     31127
40대     12695
50대      4607
60대       819
70대~      158
~10대    11540
Name: count, dtype: int64
연령대코드
~10대    0.312525
20대     0.239913
30대     0.166702
40대     0.110585
50대     0.065414
70대~    0.048962
60대     0.042462
Name: count, dtype: float64


In [ ]:
ans = df[['연령대코드','이동거리']].groupby('연령대코드').mean()
print(ans)

              이동거리
연령대코드             
20대    3211.890552
30대    3341.443859
40대    3514.857416
50대    3593.668100
60대    3538.145737
70대~   3085.039641
~10대   2634.426279


In [ ]:
target = df[(df['연령대코드'] == '20대')].reset_index(drop=True)
ans = target[target['이동거리'] >= target['이동거리'].mean()].reset_index(drop=True)
ans['탄소량'] = ans['탄소량'].astype('float')
result = ans.sort_values(['대여일자','대여소번호'], ascending=False).reset_index(drop=True).iloc[:200].탄소량
print(round(result.sum()/len(result),3))


1.613


In [ ]:
target = df[(df['대여일자'] == pd.to_datetime('2021-06-07')) & (df['연령대코드'] == '~10대')]
print(target['이용건수'].median())

1.0


In [ ]:
print(df['day_name'].unique())

['Tuesday' 'Wednesday' 'Thursday' 'Friday' 'Saturday' 'Sunday' 'Monday']


In [ ]:
target = df[(df['day_name'].isin(['Monday','Tuesday', 'Wednesday', 'Thursday', 'Friday'])) &
            (df['대여시간'].isin([6,7,8]))]
result = target.groupby(['대여시간','대여소번호']).size().to_frame('이용횟수')
ans = result.sort_values(['대여시간','이용횟수'], ascending=False).groupby('대여시간').head(3)
print(ans)

            이용횟수
대여시간 대여소번호      
8    2701    119
     646     115
     1152     92
7    259     104
     230      77
     726      77
6    2744     45
     1125     40
     1028     36


In [ ]:
ans = df[df['이동거리'] >= df['이동거리'].mean()].이동거리.std()
print(ans)

5092.139707505305


In [ ]:
df['성별'] = df.성별.map(lambda x: 'M' if x in ['M','m'] else 'F')
ans = df.groupby(['성별']).이동거리.mean()
print(ans)

성별
F    3468.575025
M    3209.110871
Name: 이동거리, dtype: float64


In [ ]:
import pandas as pd
df =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/happy2/happiness.csv',encoding='utf-8')
df.head()

,행복랭킹,나라명,점수,상대GDP,사회적지원,행복기대치,선택의 자유도,관대함,부패에 대한인식,년도
0,1,Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393,2019
1,2,Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410,2019
2,3,Norway,7.554,1.488,1.582,1.028,0.603,0.271,0.341,2019
3,4,Iceland,7.494,1.380,1.624,1.026,0.591,0.354,0.118,2019
4,5,Netherlands,7.488,1.396,1.522,0.999,0.557,0.322,0.298,2019


In [ ]:
target1 = df[df['년도'] == 2018].sort_values('점수', ascending=False).reset_index(drop=True)
target2 = df[df['년도'] == 2019].sort_values('점수', ascending=False).reset_index(drop=True)
ans1 = target1['점수'].iloc[9]
ans2 = target2['점수'].iloc[9]
print((ans1+ans2) / 2)

7.259


In [ ]:
target = df[df['행복랭킹'] <= 50][['년도','점수']].groupby('년도').mean().head()
print(target)

           점수
년도           
2018  6.64678
2019  6.67002


In [ ]:
target = df[df['년도'] == 2018]
ans = target[['점수','부패에 대한인식']].corr()
print(ans.iloc[0,1])

0.40529152271510027


In [ ]:
ans = len(df[['행복랭킹','나라명']]) - len(df[['행복랭킹','나라명']].drop_duplicates())
print(ans)

15


In [ ]:
target = df[df.년도 ==2019].drop('나라명', axis=1).corr().unstack().to_frame().reset_index(drop=False).dropna()
ans = target[target[0] != 1].sort_values(0, ascending=False).drop_duplicates(0).head(5).reset_index(drop=True)
ans.columns = ['v1','v2','corr']
print(ans)

      v1     v2      corr
0  행복기대치  상대GDP  0.835462
1  상대GDP     점수  0.793883
2  행복기대치     점수  0.779883
3  사회적지원     점수  0.777058
4  사회적지원  상대GDP  0.754906


In [ ]:
target = df.sort_values('행복랭킹', ascending=True).groupby('년도').tail(5).groupby('년도').점수.mean()
print(target)

년도
2018    3.1800
2019    3.1408
Name: 점수, dtype: float64


In [ ]:
target = df[df['년도'] == 2019]
ans1 = target[target['상대GDP'] >= target['상대GDP'].mean()]
ans2 = target[target['상대GDP'] <= target['상대GDP'].mean()]
result = ans1['점수'].mean() - ans2['점수'].mean()
print(result)

1.390483957219251


In [ ]:
target = df.sort_values(['년도','부패에 대한인식'], ascending=False)
ans = target.groupby('년도').head(20)[['년도','부패에 대한인식']].groupby('년도').mean()
print(ans)

      부패에 대한인식
년도            
2018    0.3267
2019    0.3201


In [ ]:
target1 = df[(df['년도'] == 2018) & (df['행복랭킹'] <= 50)]
target2 = df[(df['년도'] == 2019) & (df['행복랭킹'] <= 50)]

ans = len(set(target1.나라명) - set(target2.나라명))
print(ans)

4


In [184]:
cc = df.value_counts('나라명')
nn = cc[cc >= 2].index

target = df[df['나라명'].isin(nn)].reset_index(drop=True)
target.loc[target['년도']==2018,'점수'] = target[target['년도']==2018]['점수'].values * (-1)
ans = target.groupby('나라명').sum().점수.sort_values(ascending=False).iloc[0]
print(ans)

0.8700000000000001


In [49]:
#데이터 로드
x_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/airline/x_train.csv")
y_train = pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/airline/y_train.csv")
x_test= pd.read_csv("https://raw.githubusercontent.com/Datamanim/datarepo/main/airline/x_test.csv")


train = pd.merge(x_train, y_train, on = 'ID')
target = x_test.copy()
rs=810

x =train.drop(["ID",'satisfaction'], axis=1)
y = train['satisfaction']

# print(x.shape)
# print()
# print(x.info())
# print()
# print(x.describe())
# print()
# print(x.isnull().sum())
# print()
# print(target.shape)
# print()
# print(target.info())
# print()
# print(target.describe())
# print()
# print(target.isnull().sum())

obj_col = x.select_dtypes(include='object').columns
# for col in obj_col:
#   print(col)
#   print(x[col].unique())
#   print(target[col].unique())

x = pd.get_dummies(x, columns=obj_col, dtype='int')
target = pd.get_dummies(target, columns=obj_col, dtype='int')
target = target.reindex(columns=x.columns)

from sklearn.preprocessing import MinMaxScaler

num_col = x.select_dtypes(exclude='object').columns
mms = MinMaxScaler()
x[num_col] = mms.fit_transform(x[num_col])
target[num_col] = mms.transform(target[num_col])

from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x,y,test_size=0.2, stratify=y, random_state=rs)

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

rf = RandomForestClassifier(random_state=rs)

from sklearn.model_selection import GridSearchCV

param = {'n_estimators':[5,10,15,20],
         'max_depth':[5,10,15,20]}
gscv = GridSearchCV(estimator=rf, param_grid=param, cv=5)
gscv.fit(x_train, y_train)
print(gscv.best_params_)
model = gscv.best_estimator_

pred_val = model.predict(x_val)
pred_val_proba = model.predict_proba(x_val)[:,0]
pred_train = model.predict(x_train)
pred_proba_train = model.predict_proba(x_train)[:,0]


from sklearn.metrics import f1_score, roc_auc_score
print('f1_score_train:', f1_score(y_train, pred_train,pos_label = 'neutral or dissatisfied'))
print('f1_score:', f1_score(y_val, pred_val,pos_label = 'neutral or dissatisfied'))
print('auc_score_train:', roc_auc_score(y_train, pred_proba_train))
print('auc_score:', roc_auc_score(y_val, pred_val_proba))


pred = model.predict(target)

result = pd.DataFrame({'ID': x_test['ID'],
                       'pred':pred})
result.to_csv('result.csv', index=False)
print(result)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/_array_api.py:695: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmin(X, axis=axis))
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_array_api.py:712: RuntimeWarning: All-NaN slice encountered
  return xp.asarray(numpy.nanmax(X, axis=axis))


{'max_depth': 20, 'n_estimators': 20}
f1_score_train: 0.994973544973545
f1_score: 0.9666333876307078
auc_score_train: 0.0001427980694164078
auc_score: 0.007292767625293489
           ID                     pred
0           1  neutral or dissatisfied
1          16  neutral or dissatisfied
2          17  neutral or dissatisfied
3          25  neutral or dissatisfied
4          27                satisfied
...       ...                      ...
20776  103895  neutral or dissatisfied
20777  103896  neutral or dissatisfied
20778  103897                satisfied
20779  103900  neutral or dissatisfied
20780  103902  neutral or dissatisfied

[20781 rows x 2 columns]
